# Decorators

Presented at DFW Pythoneers - May 4th, 2017 - Edward Kozlowski

![Image of babushka dolls](http://legomenon.com/images/russian-matryoshka-stacking-babushka-wooden-dolls-meaning.jpg)


<img alt="easy" src="http://localhost:8000/easiest.png" style="width: 200px; height: 200px;" />

### Consider the following

In [1]:
def call_cool_function(function_x):
    function_x()

def my_cool_function():
    print("In my_cool_function")
    

call_cool_function(my_cool_function)  # <-- Note, we did not actually *call* the function.

In my_cool_function


### We can also define functions inside of functions and call them from inside the function they themselves are inside of.

In [2]:
def outer():
    print("outer")
    def inner():
        print("inner")
        print("leaving inner")
    # I call inner() here a couple times just to demonstrate that you can.
    inner()
    inner()
    print("leaving outer")

outer()

outer
inner
leaving inner
inner
leaving inner
leaving outer


### We can also return new functions from functions.  *This is the *heart* of what decorators are about*.

In [3]:
def decorate(func):
    def newfunc(*args, **kwargs):
        print("I'm in newfunc")
        result = func(*args, **kwargs)
        print("Leaving newfunc")
        return result
    return newfunc

def say_hi():
    print("Hi, my name is 'Ed'.")

def other_method():
    print("I'm another method.")
    
say_hi = decorate(say_hi)              #  <--- This is *KEY*
other_method = decorate(other_method)  #  <--- This is *KEY*

say_hi()
other_method()

I'm in newfunc
Hi, my name is 'Ed'.
Leaving newfunc
I'm in newfunc
I'm another method.
Leaving newfunc


It is ***sooo*** important to realize this is a TWO step process.

The decorate method *accepts* a function, and returns a *new* function.

### Python has syntax for calling "decorate(func)" for you.  Same as before, just with the syntatic sugar.

In [7]:
def decorate(func):
    def newfunc(*args, **kwargs):
        print("I'm in newfunc")
        result = func(*args, **kwargs)
        print("Leaving newfunc")
        return result
    return newfunc

#@decorate  # <-- This is the addition
def say_hi():
    print("Hi, my name is 'Ed'.")

#@decorate  # <-- This is the addition
def other_method():
    print("I'm another method.")
 
#I left these in here to demonstrate the syntax you can skip with the @decorate
say_hi = decorate(say_hi)
other_method = decorate(other_method)

say_hi()
other_method()

I'm in newfunc
Hi, my name is 'Ed'.
Leaving newfunc
I'm in newfunc
I'm another method.
Leaving newfunc


### An aside about *args and **kwargs

*args is just "star args", or the variable part of the arguments to a function.

**kwargs is "Keyword ARGumentS" - These are the arguments inside the signature that you're passing a name=value for.

For example:

In [11]:
def foo(*args, **kwargs):
    print("args: {}, kwargs: {}".format(repr(args), repr(kwargs)))

print("foo(1, 2, 3)")
foo(1, 2, 3)

print('\n-----------------------\n')

print("foo(one=1, two=2, three=3)")
foo(one=1, two=2, three=3)

print('\n-----------------------\n')

print("foo(1, 2, 3, one=1, two=2, three=3)")
foo(1, 2, 3, one=1, two=2, three=3)


#foo(one=1, 2, 3)

foo(1, 2, 3)
args: (1, 2, 3), kwargs: {}

-----------------------

foo(one=1, two=2, three=3)
args: (), kwargs: {'one': 1, 'two': 2, 'three': 3}

-----------------------

foo(1, 2, 3, one=1, two=2, three=3)
args: (1, 2, 3), kwargs: {'one': 1, 'two': 2, 'three': 3}


# Positional Argument Expansion


In [12]:
args = [1,2,3]
print(args)
print(*args)

def edsfunc(first, second, third):
    print("I got {} for first, {} for second, and {} for third.".format(first, second, third))

edsfunc(*args)

[1, 2, 3]
1 2 3
I got 1 for first, 2 for second, and 3 for third.


In [13]:
# What happens when we do this?

incorrect_amount_of_args = [1, 2, 3, 4]

edsfunc(*incorrect_amount_of_args)

TypeError: edsfunc() takes 3 positional arguments but 4 were given

### Print accepts any number of arguments, and prints them with a space between each argument.

In [14]:
# Print re-implemented:   ... well... kinda
import sys
def edsprint(*stuff, sep=" "):
    stuff = [str(x) for x in stuff]
    sys.stdout.write(sep.join(stuff))

edsprint('dakjjs', 3, 888, "ninety", object())

dakjjs 3 888 ninety <object object at 0x1019f4ae0>

# Keyword Argument Expansion

In [15]:
print({"Ed": 39, "Alisa": 29})


{'Ed': 39, 'Alisa': 29}


In [16]:
kwargs = {'Ed': 39, 'Alisa': 29}
print(kwargs)
print(*kwargs) #<- this just says "for key in dict, print key"
print(**kwargs)  #<- Nope, won't work.


{'Ed': 39, 'Alisa': 29}
Ed Alisa


TypeError: 'Ed' is an invalid keyword argument for this function

#### What is happening, is that we are trying to call the following:

In [17]:
print(Ed=39, Alisa=29)

TypeError: 'Ed' is an invalid keyword argument for this function

### The **var syntax means:  Expand the dictionary pointed to by "var", and use the keys as argument names, and the values as argument values.

This is an incredibly powerful part of Python, and you will see this a *lot*.

In [18]:
# Print *does* accept a keyword argument of "sep".  It controls what separates the things you're printing out.
# It defaults to space.

# Let's try this:

things = ['a','b','c','d']
arg_dict = {'sep': '*'}

print(*things, **arg_dict)

# any idea what will happen?

a*b*c*d


### Combining *args, and **kwargs lets us create "pass thrus" for functions.

In [19]:
def new_print(*args, **kwargs):
    print("Ed was here")
    print(*args, **kwargs)  # <-- Notice, we are still using stars here, and directly calling *print*.

new_print("foo")

new_print("a", 2, 34, 888, "something", object(), sep="*")

Ed was here
foo
Ed was here
a*2*34*888*something*<object object at 0x1019f4af0>


### So, what's all this got to do with Decorators then?

Let's say we wanted a new print function that yells at people instead of prints.  We *only* want it to yell in all
uppercase... Otherwise, we want to leave the functionality of "print" completely unchanged.


In [20]:
def yell(func):
    def make_args_uppercase(*args, **kwargs):
        # Now, we have to go through the *args, convert to strings, and convert them to uppercase.
        args = [str(y) for y in args]  # convert to all strings.
        args = [x.upper() for x in args] # convert to uppercase.
        func(*args, **kwargs)  # call the original function, passing the semi-original args. ;)
    return make_args_uppercase

In [21]:
# Time to have some fun.

print = yell(print)  # "Decorate" the print function.

print("I", "just", "decorated", "the", "print", "built", "in!", 888, object(), 'b', sep="~")    

I~JUST~DECORATED~THE~PRINT~BUILT~IN!~888~<OBJECT OBJECT AT 0X1019F4B00>~B


In [27]:
print("Caps lock is like cruise control for cool.")

print("Alisa")

Caps lock is like cruise control for cool.
Alisa


In [24]:
print(print)  # Well, this is meta...

<FUNCTION YELL.<LOCALS>.MAKE_ARGS_UPPERCASE AT 0X103774730>


In [25]:
__builtins__.print("hello")

hello


In [28]:
# Time to undo this madness.
print = __builtins__.print

print("please let this be in lower case!")

please let this be in lower case!


<img alt="moderate" src="http://localhost:8000/moderate.jpg" style="width: 200px; height: 200px;" />

### Side Effects of Decorators  (or "Hello @wraps!")

#### Let's say you took your time to write some really cool doc on a function.

In [29]:
def my_cool_func():
    """
    Documentation!
    """
    pass

#### Now, you can get help for your func:

In [30]:
help(my_cool_func)

Help on function my_cool_func in module __main__:

my_cool_func()
    Documentation!



#### But let's say you decorate it...

In [31]:
def my_cool_decorator(func):
    def inner(*args, **kwargs):
        func(*args, **kwargs)
    return inner

@my_cool_decorator
def my_cool_func():
    """
    Documentation!
    """
    pass

In [32]:
# So let's try to get help.
help(my_cool_func)

Help on function inner in module __main__:

inner(*args, **kwargs)



#### Several years ago when I was introduced to decorators, and learned about this:

![Picture Boy Pouts](http://reactiongifs.me/wp-content/uploads/2013/08/sad-kid-reaction-gif.gif)

#### Luckily the Python folks thought about this, and came up with a solution.  *ANOTHER* decorator!  YAY!

In [34]:
# my_cool_func take 2:

from functools import wraps

def my_cool_decorator(func):
    @wraps(func) # <---  Use of @wraps
    def inner(*args, **kwargs):
        print("in inner")
        func(*args, **kwargs)
        print("leaving inner")
    return inner

@my_cool_decorator
def my_cool_func():
    """
    Documentation!
    """
    print("my_cool_func")

In [35]:
# now let's try looking up help again.
help(my_cool_func)

Help on function my_cool_func in module __main__:

my_cool_func()
    Documentation!



### \__wrapped\__

In [36]:
print(dir(my_cool_func))


['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__wrapped__']


#### ! Wait.  What's \__wrapped\__?

In [37]:
print(my_cool_func.__wrapped__)

<function my_cool_func at 0x102313488>


#### A function!  Let's call it!

In [38]:
my_cool_func.__wrapped__()

my_cool_func


#### Hmmm.... This looks an awful lot like the original function.

####  .\__wrapped\__ is added by @wraps, and when called on the wrapped function, __wrapped__ provides access to the original "undecorated" function.
####  Hmm...

In [39]:
print(my_cool_func.__wrapped__.__wrapped__)

AttributeError: 'function' object has no attribute '__wrapped__'

#### Oops, too far.

#### What happens if we wrap it more than once?

#### my_cool_func take 3!

In [40]:
from functools import wraps

def my_cool_decorator(func):
    @wraps(func) # <---  Use of @wraps
    def inner(*args, **kwargs):
        print("my cool decorator")
        func(*args, **kwargs)
    return inner

def my_other_cool_decorator(func):
    @wraps(func) # <---  Use of @wraps
    def inner(*args, **kwargs):
        print("my other cool decorator")
        func(*args, **kwargs)
    return inner

@my_other_cool_decorator
@my_cool_decorator
def my_cool_func(name="Ed"):
    """
    Documentation!
    """
    print("My cool func")

# Now, running this:
my_cool_func()

my other cool decorator
my cool decorator
My cool func


In [41]:
print(my_cool_func)
my_cool_func()

print(my_cool_func.__wrapped__)
my_cool_func.__wrapped__()

print(my_cool_func.__wrapped__.__wrapped__)
my_cool_func.__wrapped__.__wrapped__()

# does my doc still work?
help(my_cool_func)

<function my_cool_func at 0x1037ecbf8>
my other cool decorator
my cool decorator
My cool func
<function my_cool_func at 0x1037ecb70>
my cool decorator
My cool func
<function my_cool_func at 0x1037ecae8>
My cool func
Help on function my_cool_func in module __main__:

my_cool_func(name='Ed')
    Documentation!



# HERE BE DRAGONS!

<img alt="Dragons" src="http://2.bp.blogspot.com/_D9a8fe6Tue8/RvIZrkGLDvI/AAAAAAAAABs/u8HV6yXGqBM/s400/Dragonpotter.bmp" style="width: 300px; height: 300px;" />

* This is new in Python 3.4.  Prior to this, calls to func.\__wrapped\__ would just return the original function completely unwrapped, skipping all wrapping levels, or in some cases, not work at all.  *Looking at you 2.x series.*

Essentially, `@wraps` copies the function metadata (func.\__name\__, func.\__doc\__, ... etc...) and applies it to the wrapped function, so we don't have to.  This also includes the function's signature.

In [42]:
from inspect import signature
print(signature(my_cool_func))


(name='Ed')


In [43]:
# Let's define it without wraps first, and make it take an argument.
def my_cool_decorator(func):
    def inner(*args, **kwargs):
        func(*args, **kwargs)
    return inner

@my_cool_decorator
def my_cool_func(my_arg=None):
    """
    Documentation!
    """
    pass

In [44]:
print(signature(my_cool_func))

(*args, **kwargs)


In [45]:
# Now with wraps
def my_cool_decorator(func):
    @wraps(func)
    def inner(*args, **kwargs):
        func(*args, **kwargs)
    return inner

@my_cool_decorator
def my_cool_func(my_arg=None):
    """
    Documentation!
    """
    pass

In [46]:
print(signature(my_cool_func))

(my_arg=None)


<img alt="most_difficult" src="http://localhost:8000/most_difficult.png" style="width: 200px; height: 200px;" />

# Decorators That Take Arguments

In [53]:
# Let's say we want to write a decorator that prepends and appends a user-defined html tag to the output of a function.
from functools import wraps

def htmltags(tag=None):  # <-- Why is this not wrong.  Shouldn't we accept a func?  ;)
    def decorate(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            return "<{tag}>\n{out}</{tag}>\n".format(tag=tag, out=func(*args, **kwargs))
        return wrapper
    return decorate

@htmltags(tag="somethign_else")
def my_paragraph():
    return "This is in a para tag\n"

print(my_paragraph())


<somethign_else>
This is in a para tag
</somethign_else>



In [54]:
# Going crazy.  HTML page with decorators with arguments.

@htmltags('html')
@htmltags('body')
@htmltags('div')
@htmltags('h1')
def my_page():
    return "Hello World!\n"

print(my_page())

<html>
<body>
<div>
<h1>
Hello World!
</h1>
</div>
</body>
</html>



## Dustier Corners

<img alt="expert" src="http://localhost:8000/expert.png" style="width: 200px; height: 200px;" />


### Optional Arguments to Decorators

In [55]:
# The naive way:
from functools import wraps

def htmltags(func=None, tag="Default"):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return "<{tag}>\n{out}</{tag}>\n".format(tag=tag, out=func(*args, **kwargs))
    return wrapper

@htmltags(tag='p')
def my_paragraph():
    return "This is a paragraph\n"

@htmltags
def my_default():
    return "This is a default tag\n"

print(my_default())
print(my_paragraph())

TypeError: 'NoneType' object is not callable

### Now, I struggled with this one a *lot*!  Apparently, when decorators are created, the initial call to the decorator is passed with the function being decorated being "None".  This is because when we put parens after the @decorator, Python assumes we are passing *all* of the arguments to construct the decorator.

We have to recall the calling order.

```
@htmltags(tag=p)
def my_paragraph():
```

Translates to:

```my_paragraph = htmltags(func=None, tag=p)(my_paragraph)```

In other words, htmltags(func=None, tag=p) gets evaluated **first**, the decorator (wrapper) never sees "func", and the wrapper fails, because indeed, the NoneType 'func' is not callable, and the function fails.

To get around this, we can use a partial to construct a decorator from our decorator so that our optional arguments are evaulated.

In [63]:
# The "proper" way

from functools import wraps
from functools import partial

def htmltags(func=None, tag="Default"):
    if func is None:
        return partial(htmltags, tag=tag)  
    @wraps(func)
    def wrapper(*args, **kwargs):
        return "<{tag}>\n{out}</{tag}>\n".format(tag=tag, out=func(*args, **kwargs))
    return wrapper

@htmltags(tag='p')
def my_paragraph():
    return "This is a paragraph\n"

@htmltags
def my_default():
    return "This is a default tag\n"

print(my_default())
print(my_paragraph())


<Default>
This is a default tag
</Default>

<p>
This is a paragraph
</p>



In [57]:
from operator import mul
double = partial(mul, 2)

print(double(4))

8


#### Decorating our Decorator

This code:

```
@htmltags
def my_default():
```

Is easy to understand.  It becomes:

```
htmltags(my_default)()
```

In this code:

```
@htmltags(tag='p')
def my_paragraph()
```

Translates to:

```
my_paragraph = htmltags(func=None, tag='p')(my_paragraph)()
```

The section:

```
htmltags(func=None, tag='p')
```

Is evaluated first, to try and construct the decorator.  This time, since func is None, we have to return a partial from the first call.  The partial is the htmltags function itself, which constructs *another* decorator with the tag defaulted to what we passed in.

So we get:

```
my_paragraph = htmltags(my_paragraph)
```

Except *this* htmltags decorator defaults the tag as we had specified.


# Decorators With Classes

<img src="https://pbs.twimg.com/profile_images/419232639246020608/sVo1VbHb.jpeg" alt="Kid with Decorated Class" style="width: 350px; height: 300px"/>

### Classmethods

A classmethod must have a reference to a class as its first parameter.  Not a reference to "self", (the instance), but a reference to a *class*.

Classmethods can be useful for creating alternate constructors for classes.

In [66]:
# Ex:

isbns = {
    1: ("The Hobbit", "J.R.R. Tolkien"),
    2: ("Armor", "John Steakley")
}

class Book:
    
    def __init__(self, title, author):
        self.title = title
        self.author = author
    
    def print_title(self):
        print("{} by {}".format(self.title, self.author))
    
    @classmethod
    def from_isbn(cls, isbn):
        # Lookup title and author from isbn:
        title, author = isbns.get(isbn)
        # return an instance of this class.
        return cls(title=title, author=author)  # Notice, we're saying cls(title=...) and not self(title=...)

Book(title="The Stand", author="Stephen King").print_title()

# semi-equivalent to from_isbn(Book, 2)... but not exactly.
b_obj = Book.from_isbn(2)
b_obj.print_title()

Book.from_isbn(1).print_title()

The Stand by Stephen King
Armor by John Steakley
The Hobbit by J.R.R. Tolkien


### Staticmethods

A Staticmethod does not take either a class reference, or an instance reference as its first parameter.  It can be referred to by both the `self.my_static_method(args)` syntax, as well as the `Class.my_static_method(args)` syntax.

The caveat here is that static methods have no access to the instances that they're used in.  (You lose access to 'self').

This is useful if you have a method that logically belongs with a class, but doesn't necessarily require the instantiation of the class.

In [70]:
# Back to our library example:

isbns = {
    1: ("The Hobbit", "J.R.R. Tolkien"),
    2: ("Armor", "John Steakley")
}

class Book:
    
    def __init__(self, title, author):
        self.title = title
        self.author = author
        
    def print_title(self):
        print("{} by {}".format(self.title, self.author))
    
    @classmethod
    def from_isbn(cls, isbn):
        # Lookup title and author from isbn:
        title, author = isbns.get(isbn)
        # return an instance of this class.
        return cls(title=title, author=author)
    
    @staticmethod
    def is_valid_isbn(isbn):
        # I realize *real* ISBN's actually HAVE a valid format... this is just a trivial example.
        if isinstance(isbn, int):
            return True
        return False

# Now we can use the is_valid_isbn both inside and outside the Book instance.
print(Book.is_valid_isbn(2))
b = Book.from_isbn(2)
print(b.is_valid_isbn(1))

True
True


In [73]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):  # <-- We need to provide 'self' here since we're part of the class.
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("Decorator 1")
            return func(*args, **kwargs)
        return wrapper
    
    # Decorator as a classmethod
    @classmethod
    def decorator2(cls, func):  # <-- We need to provide 'cls' since we're a classmethod.
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("Decorator 2")
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a staticmethod
    @staticmethod
    def decorator3(func):  # <-- We need to provide 'cls' since we're a classmethod.
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("Decorator 3")
            return func(*args, **kwargs)
        return wrapper
# As an instance method:
a = A()

# As a regular method
@a.decorator1
def spam():
    print("Spam!")
    
#As a class method
@A.decorator2
def grok():
    print("Grok!")

# As a static method
@a.decorator3
def foo():
    print("foo")

@A.decorator3
def bar():
    print("bar")

spam()
grok()
foo()
bar()

        

Decorator 1
Spam!
Decorator 2
Grok!
Decorator 3
foo
Decorator 3
bar


In [77]:
import random
my_list = list(range(1,20))
random.shuffle(my_list)
print(my_list.pop())

2


In [ ]:
# Applying class decorators with inheritance:

class B(A):

    # We call '@a' because decorator 1 is defined as a method in class 'A', and as such, is *only* accessible from
    # the 'a' instance of class 'A'.
    @a.decorator1
    def foo(self):
        print("Foo!")
        
    # We call '@A' here because decorator 2 is defined as a classmethod, and as such is accessible as an attribute of
    # the 'A' class.
    @A.decorator2
    def bar(self):
        print("Bar!")

b_obj = B()
b_obj.foo()
b_obj.bar()

## Decorators *as* classes

In [ ]:
# To implement a decorator as a class, you have to implement both the __call__ and the __get__ methods.

# This is where I was in my presentation when I ran out of time.  This does work, and we can discuss.

import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
    
    def __call__(self, *args, **kwargs):
        print("Call called")
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)

    # Whenever functions implementing methods are looked up in a class, their __get__ method is invoked.
    # The purpose of __get__ is to provide a bound method object, which ultimately supplies "self" to the method.
    def __get__(self, instance, cls):
        print("Get called!")
        if instance is None:
            print("Instance is none")
            return self
        else:
            print("Instance is {}".format(instance))
            return types.MethodType(self, instance)

# usage:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

add(2,3)
add(4,5)
print("add calls: {}".format(add.ncalls))

s = Spam()
s.bar(1)
s.bar(2)
s.bar(3)
print("Spam.bar calls: {}".format(Spam.bar.ncalls))  # <-- "Instance is none"  (we're referencing the class directly)


    

### One of my favorite decorators

In [ ]:
class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def say_hi(self):
        print("My name is {} and I am {} years old.".format(self.name, self.age))
        
me = Person('Ed', 39)
wife = Person('Alisa', 41)  # I'm a dead man.

me.say_hi()
wife.say_hi()

wife.age = 29  # Much better.

me.say_hi()
wife.say_hi()

In [ ]:
# Let's convert Person to be a little smarter about my wife's age.  She's never a day over 29.  :)

class Person:
    
    def __init__(self, name, age):
        self.name = name
        self._age = age    # <--- This changed.

    @property                                    # <--- Added
    def age(self):                               #   |
        if self.name == "Alisa": return 29       #   | *wink* *wink* *nudge* *nudge*
        return self._age                         #   |
    
    @age.setter                                  # <-- We added this too.
    def age(self, age):                          #   |
        if self.name == 'Alisa' and age > 29:    #   |
            self._age = 29                       #   |  Save my life.
        else:                                    #   | 
            self._age = age                      #   |

#------------------------------------------------------------------------------------------------
# Extra special note!
# From here below stayed *exactly* the same.  I'm not calling person.age_setter(age).  :)
#------------------------------------------------------------------------------------------------

    def say_hi(self):
        print("My name is {} and I am {} years old.".format(self.name, self.age))
        
me = Person('Ed', 39)
wife = Person('Alisa', 41)  # I'm a dead man...  Or am I?

me.say_hi()
wife.say_hi()

wife.age = 29  # Much better.

me.say_hi()
wife.say_hi()

# Credits

- David Beazley & Brian K. Jones - Python Cookbook version 3  (Without which I would never have understood the use of partial to decorate the decorator with optional arguments.)

